# 決定木のアルゴリズム

<img src='desitiontree.png'>

## 目的関数

### ジニ不純度

$$
I_G(t) = 1 - \sum_{i=1}^c p(i|t)^2
$$

これを目的関数にとる。

これが大きいほどそのノード内にいろんなものがごちゃごちゃしていて、小さいほどそのノード内が整理されていることを示す。

### 情報利得

$$
IG(D_p, f) = I_G(D_p) - \frac{N_{left}}{N_p} I_G(D_{left}) - \frac{N_{right}}{N_p} I_G(D_{right}) 
$$

$
親ノード D_p => 子ノード D_{left}, D_{right} と分ける\\
親ノード内のトレーニングサンプルの数: N_p\\
子ノード内のトレーニングサンプルの数をそれぞれN_{left}、N_{right}\\
分割する特徴量: f
$

つまり　**親の不純度 - 子の不純度** ってこと！

これが大きい質問が**良い質問**、ということになる。

過学習を起こしやすいので、ノードの深さの上限を指定したり、ノード分けでだんだん小さくなっていく (情報利得) × (ノード内のデータ数割合) に閾値を設けたりする。

また、他のモデルとして、ランダムフォレストや勾配ブースティング木などのアンサンブルも存在する（今度できたら？）

### 説明変数の重要度

決定木がよく使われる理由の一つに説明変数の重要度を出力できる、と言うことがある。

情報利得が最大になるようにで分割するので、全情報利得のうちの、その説明変数が占める割合で表される。

$$
FI(F_j) = \sum_{t \in N_{f_j}}IG(t,j_j)n_t
$$

$
j番目の特徴量f_jで分割されたノードの集合:n_{f_j}\\
N_{f_j}に含まれるノードtのサンプル数: n_t\\
f_jの重要度：FI(f_j)
$

正規化された特徴量は、


$$
FI_n(f_j) = \frac{FI_(f_j)}{\sum_{j=1}^{n} FI(f_j)}
$$

と表される

## 具体的なアルゴリズム

1. 特徴量の１つに注目し、その値を昇順に並べ、各隣り合う値の中間値で分けて行き、情報利得を参照していく
2. その中で情報利得が最大になる分け方を選択(もし同じになればランダム)
3. これを繰り返していく

# 実装

## 方針

機能は以下の通り

1. fit(data, target)で学習
2. predict(data)で予測
3. score(data, target)で正答率
4. feature\_importances\_で重要度を出す

再帰関数で書いていく

<font size=1px>（オブジェクト指向っぽくかけたらいいな。）</font>

_NodeクラスとDecisionTreeクラスと_TreeAnalysiusの3つのクラスからなる。

決定木自体の過学習回避には、(情報利得) × (ノード内のデータ数割合)　の閾値設定とmax_depthの二つがあるが、今回は両方実装する!

## 実装！

In [1]:
import numpy as np

In [2]:
class _Node:
        #data, target は np.array
        def __init__(self, data, max_depth, random_state, standard):
            self.right = None
            self.left = None
            self.max_depth = max_depth
            self.random_state = random_state
            self.n_features = data.shape[1]
            self.label = None
            self.standard = None
            self.best_feature_idx = None
            self.best_threshold = None
        # 子ノード(left, right)に分ける
        def split(self, data, target, depth):
            self.depth = depth
            
            #自ノードの分析
            class_cnt = {i: len(target[target==i]) for i in np.unique(target)}
            self.label = max((value, key) for (key, value) in class_cnt.items())[1]
            
            self.label = target[0]
            if depth == self.max_depth:
                return
            self.gini_idx = self.gini(target)
            
            #各情報利得を計算し、最大のものを記憶
            self.best_info_gain = 0
            
            if self.random_state is not None:
                np.random.seed(self.random_state)
            loop_order = np.random.permutation(self.n_features)
            for i in loop_order:
                unique_features = np.unique(data[:, i])
                self.thresholds = (unique_features[:-1] + unique_features[1:]) / 2
                for threshold in self.thresholds:
                    child_left = target[data[:, i] < threshold]
                    child_right = target[data[:, i] >= threshold]
                    info_gain = self.info_gain(child_left, child_right, target)
                    
                    if info_gain > self.best_info_gain:
                        self.best_info_gain = info_gain
                        self.best_feature_idx = i
                        self.best_threshold = threshold
                        self.child_left = child_left
                        self.child_right = child_right
                        
            if not(self.best_info_gain):
                return
            
            # 再帰させる
            data_left = data[data[:, self.best_feature_idx] < self.best_threshold]
            data_right = data[data[:, self.best_feature_idx] >= self.best_threshold]
            
            target_left = target[data[:, self.best_feature_idx] < self.best_threshold]
            target_right = target[data[:, self.best_feature_idx] >= self.best_threshold]
            
            self.left = _Node(data_left, self.max_depth, self.random_state, self.standard)
            self.right = _Node(data_right, self.max_depth, self.random_state, self.standard)
            
            self.left.split(data_left, target_left, self.depth+1)
            self.right.split(data_right, target_right, self.depth+1)
            
        def predict(self, data):
            if not(self.best_info_gain):
                return self.label
            if data[self.best_feature_idx] < self.best_threshold:
                return self.left.predict(data)
            return self.right.predict(data)
            
        def gini(self, target):
            sigma = 0
            for i in np.unique(target):
                sigma += (len(target[target==i]) / len(target)) ** 2
            return (1 - sigma)

        def info_gain(self, left, right, target):
            return (self.gini_idx - len(left) / len(target) * self.gini(left) - len(right) / len(target) * self.gini(right))

        def trim(self, standard):
            pass

In [3]:
class MyDecisionTree:
    def __init__(self, max_depth=None, random_state=None, standard=None):
        self.max_depth = max_depth
        self.random_state = random_state
        self.standard = standard
        
    def fit(self, data, target):
        self.root = _Node(data, self.max_depth, self.random_state, self.standard)
        self.root.split(data, target, 0)
        self.root.trim(self.standard)
        self.feature_importances = None
        
    def  predict(self, data):
        ans = []
        if len(data.shape) == 1:
            return self.root.predict(data)
        for d in data:
            ans.append(self.root.predict(d))
        return np.array(ans)
    
    def score(self, data, target):
        pred = self.predict(data)
        return(len(pred[pred==target]) / len(target))

# 実際に使ってみる

sklearnのDecisionTreeClassifierと比較する。

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## アヤメ

In [5]:
from sklearn.datasets import load_iris
iris = load_iris()

In [6]:
X = iris.data
y = iris.target

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
mdt = MyDecisionTree(random_state=0)

In [9]:
mdt.fit(X_train, y_train)

In [10]:
print(mdt.score(X_train,y_train))
print(mdt.score(X_test, y_test))

1.0
0.9777777777777777


In [11]:
dtc = DecisionTreeClassifier(random_state=0)

In [12]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [13]:
print(dtc.score(X_train,y_train))
print(dtc.score(X_test, y_test))

1.0
0.9777777777777777


一緒！！

# 課題、感想等

そもそも仕組みが単純なモデルであること、有能な参考文献が手に入ったことで実装することができた。

実際はジニ係数の他に、引数によって情報エントロピーも取れるので、その実装もできればよかった。

ランダムフォレストの実装もできたらいいな、、、と思う。